In [1]:
#Api Get Method - Stock Ledger Entry - Mahape
import requests
import pandas as pd
import time 
start_time = time.time()

base_url = 'https://erpv14.electrolabgroup.com/'
endpoint = 'api/resource/Stock Ledger Entry'
url = base_url + endpoint

params = {
    'fields': '["item_code","warehouse","qty_after_transaction","posting_datetime"]',
    'limit_start': 0, 
    'limit_page_length': 1000000000000000,
    'filters': '[["warehouse", "=", "Mahape - EIPL"]]'
}

headers = {
    'Authorization': 'token 3ee8d03949516d0:6baa361266cf807'
}
response = requests.get(url, params=params, headers=headers)

if response.status_code == 200:
    data = response.json()
    print("Fields are correct.")
    mahape_ledger = pd.DataFrame(data['data'])
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
    print("Response:", response.json())


Fields are correct.


In [2]:
mahape_ledger.head()

,item_code,warehouse,qty_after_transaction,posting_datetime
0,0633000007/00,Mahape - EIPL,0.0,2024-09-26 10:35:58
1,IC DS 1307,Mahape - EIPL,226.0,2024-09-27 12:40:29
2,BUZZER B 27 WITH 3 WIRES,Mahape - EIPL,0.0,2024-09-26 10:35:58
3,0115 Leg Bush 28 mm,Mahape - EIPL,1120.0,2024-09-27 12:40:29
4,ELPCBASM06X2305001I,Mahape - EIPL,0.0,2024-09-26 10:35:58


In [3]:
mahape_ledger_sorted = mahape_ledger.sort_values(by='posting_datetime', ascending = False)

mahape_ledger_unique = mahape_ledger_sorted.drop_duplicates(subset='item_code', keep='first')
mahape_ledger_unique.head()

,item_code,warehouse,qty_after_transaction,posting_datetime
29,Peroxide Cured silicon Tubing ID 6mm x OD 9mm,Mahape - EIPL,248.0,2024-09-27 12:48:12
10,ELPCBASM2310004,Mahape - EIPL,6.0,2024-09-27 12:48:00
82,PVC INSULATED SHIELDED CABLE 4 CORE,Mahape - EIPL,160.0,2024-09-27 12:43:57
32,ELPCBASM06X2305001I,Mahape - EIPL,33.0,2024-09-27 12:40:29
33,0633000007/00,Mahape - EIPL,29.0,2024-09-27 12:40:29


In [4]:
#Api Get Method - Mahape Item  -  1 
import requests
import pandas as pd
import time 
start_time = time.time()

base_url = 'https://erpv14.electrolabgroup.com/'
endpoint = 'api/resource/Item'
url = base_url + endpoint

params = {
    'fields': '["item_name","item_code","item_defaults.default_warehouse"]',
    'limit_start': 0, 
    'limit_page_length': 10000000000000,
    'filters': '[["disabled", "=", "No"], ["item_name", "not like", "%unprocessed%"]]'
}

headers = {
    'Authorization': 'token 3ee8d03949516d0:6baa361266cf807'
}
response = requests.get(url, params=params, headers=headers)

if response.status_code == 200:
    data = response.json()
    print("Fields are correct.")
    mahape_item_df = pd.DataFrame(data['data'])
    mahape_item_df = mahape_item_df[mahape_item_df['default_warehouse'] == 'Mahape - EIPL']
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
    print("Response:", response.json())


Fields are correct.


In [5]:
mahape_item_df.head()

,item_name,item_code,default_warehouse
24,(0901)EH-01A Power Adapter With Cable,(0901)EH-01A Power Adapter With Cable,Mahape - EIPL
26,(OLD) 0182 Bath Outlet Nozzle 88/00,(OLD) 0182000088/00,Mahape - EIPL
27,(OLD) 0213 Main Plate with Clinch Nut Assembly...,"(OLD) 0213000017/01,16/02",Mahape - EIPL
32,(OLD) 0913 Width Module Opto Cutter 50/01,(OLD) 0913000050/01,Mahape - EIPL
34,(OLD) 0913 Base Plate 123/04,(OLD) 0913000123/04,Mahape - EIPL


In [6]:
mahape = pd.merge(mahape_item_df,mahape_ledger_unique , on='item_code', how='outer')
mahape.rename(columns={'qty_after_transaction': 'Mahape Quantity'}, inplace=True)
mahape

,item_name,item_code,default_warehouse,warehouse,Mahape Quantity,posting_datetime
0,NaN,"""z"" Stand (M.S)",NaN,Mahape - EIPL,11.0,2021-09-07 14:06:46
1,NaN,( 0130000086/02 ),NaN,Mahape - EIPL,1.0,2024-08-06 12:03:44
2,NaN,( 0207000182/00 ),NaN,Mahape - EIPL,0.0,2024-03-05 10:19:57
3,NaN,( R & D) 0115 Square Pillar for Opto,NaN,Mahape - EIPL,2.0,2020-08-19 11:00:00
4,(0901)EH-01A Power Adapter With Cable,(0901)EH-01A Power Adapter With Cable,Mahape - EIPL,Mahape - EIPL,0.0,2024-09-04 15:45:26
...,...,...,...,...,...,...
20478,pH Buffer Solution 4.00,pH Buffer Solution 4.00,Mahape - EIPL,NaN,NaN,NaN
20479,pH Buffer Solution 7.00,pH Buffer Solution 7.00,Mahape - EIPL,NaN,NaN,NaN
20480,NaN,packing tapr dispenser,NaN,Mahape - EIPL,0.0,2024-07-29 14:07:44
20481,3213,"signboard painter's colours, modifying tints, ...",Mahape - EIPL,NaN,NaN,NaN


In [7]:
##-----------------------------------  Quality Control


In [8]:
#Api Get Method - Stock Ledger Entry - Quality Control
import requests
import pandas as pd
import time 
start_time = time.time()

base_url = 'https://erpv14.electrolabgroup.com/'
endpoint = 'api/resource/Stock Ledger Entry'
url = base_url + endpoint

params = {
    'fields': '["item_code","warehouse","qty_after_transaction","posting_datetime"]',
    'limit_start': 0, 
    'limit_page_length': 1000000000000000,
    'filters': '[["warehouse", "=", "Quality Control - EIPL"]]'
}

headers = {
    'Authorization': 'token 3ee8d03949516d0:6baa361266cf807'
}
response = requests.get(url, params=params, headers=headers)

if response.status_code == 200:
    data = response.json()
    print("Fields are correct.")
    qc_ledger = pd.DataFrame(data['data'])
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
    print("Response:", response.json())


Fields are correct.


In [9]:
qc_ledger.head()

,item_code,warehouse,qty_after_transaction,posting_datetime
0,( 0130000086/02 ),Quality Control - EIPL,30.0,2020-06-30 12:14:12
1,( 0130000086/02 ),Quality Control - EIPL,60.0,2020-08-13 16:36:45
2,( 0130000086/02 ),Quality Control - EIPL,1.0,2020-08-24 15:27:06
3,( 0130000086/02 ),Quality Control - EIPL,0.0,2020-08-31 14:59:16
4,( 0130000086/02 ),Quality Control - EIPL,0.0,2020-06-30 12:15:00


In [10]:
qc_ledger_sorted = qc_ledger.sort_values(by='posting_datetime', ascending = False)

qc_ledger_unique = qc_ledger_sorted.drop_duplicates(subset='item_code', keep='first')
qc_ledger_unique.head()

,item_code,warehouse,qty_after_transaction,posting_datetime
122553,"Stepper Motor 23H-76-28-4A, 20Kgcm",Quality Control - EIPL,3.0,2024-09-27 12:29:04
60258,0701 -Air Valve Mtg Plate,Quality Control - EIPL,20.0,2024-09-27 11:59:56
80478,1601000004/00 Unprocessed,Quality Control - EIPL,0.0,2024-09-27 11:55:35
80419,1601000003/00 Unprocessed,Quality Control - EIPL,0.0,2024-09-27 11:55:35
31325,0173000006/04,Quality Control - EIPL,50.0,2024-09-27 11:50:01


In [11]:
#Api Get Method - Quality Control - 2
import requests
import pandas as pd
import time 
start_time = time.time()

base_url = 'https://erpv14.electrolabgroup.com/'
endpoint = 'api/resource/Item'
url = base_url + endpoint

params = {
    'fields': '["item_name","item_code","item_defaults.default_warehouse"]',
    'limit_start': 0, 
    'limit_page_length': 10000000000000,
    'filters': '[["disabled", "=", "No"]]'
}

headers = {
    'Authorization': 'token 3ee8d03949516d0:6baa361266cf807'
}
response = requests.get(url, params=params, headers=headers)

if response.status_code == 200:
    data = response.json()
    print("Fields are correct.")
    qc_item_df = pd.DataFrame(data['data'])
    qc_item_df = qc_item_df[qc_item_df['default_warehouse'] == 'Quality Control - EIPL']
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
    print("Response:", response.json())


Fields are correct.


In [12]:
qc_item_df

,item_name,item_code,default_warehouse


In [13]:
quality_control = pd.merge(qc_item_df,qc_ledger_unique , on='item_code', how='right')
quality_control.rename(columns={'qty_after_transaction': 'QualityControl Quantity'}, inplace=True)
quality_control

,item_name,item_code,default_warehouse,warehouse,QualityControl Quantity,posting_datetime
0,NaN,"Stepper Motor 23H-76-28-4A, 20Kgcm",NaN,Quality Control - EIPL,3.0,2024-09-27 12:29:04
1,NaN,0701 -Air Valve Mtg Plate,NaN,Quality Control - EIPL,20.0,2024-09-27 11:59:56
2,NaN,1601000004/00 Unprocessed,NaN,Quality Control - EIPL,0.0,2024-09-27 11:55:35
3,NaN,1601000003/00 Unprocessed,NaN,Quality Control - EIPL,0.0,2024-09-27 11:55:35
4,NaN,0173000006/04,NaN,Quality Control - EIPL,50.0,2024-09-27 11:50:01
...,...,...,...,...,...,...
7825,NaN,0806 250 Ml HDPE Chuck Assy,NaN,Quality Control - EIPL,0.0,2020-03-03 14:53:07
7826,NaN,Loctite AA 349,NaN,Quality Control - EIPL,0.0,2020-03-03 12:07:29
7827,NaN,Digital Micrometer Mitutoyo 0- 25 mm,NaN,Quality Control - EIPL,0.0,2020-03-03 11:43:08
7828,NaN,1602000044/00,NaN,Quality Control - EIPL,0.0,2020-03-03 11:32:18


In [14]:
merged_df = pd.merge(mahape,quality_control , on='item_code', how='outer')
merged_df['Mahape Quantity'] = merged_df['Mahape Quantity'].fillna(0)
merged_df['QualityControl Quantity'] = merged_df['QualityControl Quantity'].fillna(0)
merged_df['Stock Mahape + QC'] = merged_df['Mahape Quantity'] + merged_df['QualityControl Quantity']

merged_df.head()

,item_name_x,item_code,default_warehouse_x,warehouse_x,Mahape Quantity,posting_datetime_x,item_name_y,default_warehouse_y,warehouse_y,QualityControl Quantity,posting_datetime_y,Stock Mahape + QC
0,NaN,"""z"" Stand (M.S)",NaN,Mahape - EIPL,11.0,2021-09-07 14:06:46,NaN,NaN,NaN,0.0,NaN,11.0
1,NaN,( 0130000086/02 ),NaN,Mahape - EIPL,1.0,2024-08-06 12:03:44,NaN,NaN,Quality Control - EIPL,0.0,2023-11-04 13:40:56,1.0
2,NaN,( 0207000182/00 ),NaN,Mahape - EIPL,0.0,2024-03-05 10:19:57,NaN,NaN,Quality Control - EIPL,0.0,2023-12-01 14:18:47,0.0
3,NaN,( R & D) 0115 Square Pillar for Opto,NaN,Mahape - EIPL,2.0,2020-08-19 11:00:00,NaN,NaN,NaN,0.0,NaN,2.0
4,(0901)EH-01A Power Adapter With Cable,(0901)EH-01A Power Adapter With Cable,Mahape - EIPL,Mahape - EIPL,0.0,2024-09-04 15:45:26,NaN,NaN,NaN,0.0,NaN,0.0


In [15]:
merged_df = merged_df[['item_code', 'Mahape Quantity', 'QualityControl Quantity', 'Stock Mahape + QC']]
merged_df

,item_code,Mahape Quantity,QualityControl Quantity,Stock Mahape + QC
0,"""z"" Stand (M.S)",11.0,0.0,11.0
1,( 0130000086/02 ),1.0,0.0,1.0
2,( 0207000182/00 ),0.0,0.0,0.0
3,( R & D) 0115 Square Pillar for Opto,2.0,0.0,2.0
4,(0901)EH-01A Power Adapter With Cable,0.0,0.0,0.0
...,...,...,...,...
20652,pH Buffer Solution 4.00,0.0,0.0,0.0
20653,pH Buffer Solution 7.00,0.0,0.0,0.0
20654,packing tapr dispenser,0.0,0.0,0.0
20655,"signboard painter's colours, modifying tints, ...",0.0,0.0,0.0


In [16]:
########---------


In [17]:
#Api Get Method - purchase order -1
import requests
import pandas as pd
import time 
start_time = time.time()

base_url = 'https://erpv14.electrolabgroup.com/'
endpoint = 'api/resource/Purchase Order'
url = base_url + endpoint

params = {
    'fields': '["items.item_code","items.qty","items.received_qty"]',
    'limit_start': 0, 
    'limit_page_length': 1000000000000,
    'filters': '[["status", "IN", ["To Receive and Bill", "To Receive"]]]'
}

headers = {
    'Authorization': 'token 3ee8d03949516d0:6baa361266cf807'
}
response = requests.get(url, params=params, headers=headers)

if response.status_code == 200:
    data = response.json()
    print("Fields are correct.")
    rnb_df = pd.DataFrame(data['data'])
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
    print("Response:", response.json())


Fields are correct.


In [18]:
rnb_df = rnb_df.groupby('item_code').sum()

rnb_df.head()

,qty,received_qty
item_code,,
(OLD) 0903000040/02,50.0,3.0
(OLD) 0903000139/02,11.0,0.0
(OLD) 0913000027/00,10.0,10.0
0104 Corru. Box Glass Jar Printed) 3 Ply 152 X 152 X 215mm,2000.0,0.0
0104 Customized Basket Lid 80 Mesh Dia 12 mm,20.0,0.0


In [19]:
rnb_df['Pending Qty'] = rnb_df['qty'] - rnb_df['received_qty']
rnb_df.head()

,qty,received_qty,Pending Qty
item_code,,,
(OLD) 0903000040/02,50.0,3.0,47.0
(OLD) 0903000139/02,11.0,0.0,11.0
(OLD) 0913000027/00,10.0,10.0,0.0
0104 Corru. Box Glass Jar Printed) 3 Ply 152 X 152 X 215mm,2000.0,0.0,2000.0
0104 Customized Basket Lid 80 Mesh Dia 12 mm,20.0,0.0,20.0


In [20]:
rnb_df = rnb_df.reset_index()
rnb_df = rnb_df[['item_code', 'Pending Qty']]
rnb_df.head()

,item_code,Pending Qty
0,(OLD) 0903000040/02,47.0
1,(OLD) 0903000139/02,11.0
2,(OLD) 0913000027/00,0.0
3,0104 Corru. Box Glass Jar Printed) 3 Ply 152 X...,2000.0
4,0104 Customized Basket Lid 80 Mesh Dia 12 mm,20.0


In [21]:
result_df = pd.merge(merged_df,rnb_df, on='item_code', how='outer')
result_df.head()

,item_code,Mahape Quantity,QualityControl Quantity,Stock Mahape + QC,Pending Qty
0,"""z"" Stand (M.S)",11.0,0.0,11.0,NaN
1,( 0130000086/02 ),1.0,0.0,1.0,NaN
2,( 0207000182/00 ),0.0,0.0,0.0,NaN
3,( R & D) 0115 Square Pillar for Opto,2.0,0.0,2.0,NaN
4,(0901)EH-01A Power Adapter With Cable,0.0,0.0,0.0,NaN


In [22]:
result_df['Pending Qty'] = result_df['Pending Qty'].fillna(0)
result_df['Total'] = result_df['Stock Mahape + QC'] + result_df['Pending Qty']

result_df

,item_code,Mahape Quantity,QualityControl Quantity,Stock Mahape + QC,Pending Qty,Total
0,"""z"" Stand (M.S)",11.0,0.0,11.0,0.0,11.0
1,( 0130000086/02 ),1.0,0.0,1.0,0.0,1.0
2,( 0207000182/00 ),0.0,0.0,0.0,0.0,0.0
3,( R & D) 0115 Square Pillar for Opto,2.0,0.0,2.0,0.0,2.0
4,(0901)EH-01A Power Adapter With Cable,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
20722,pH Buffer Solution 4.00,0.0,0.0,0.0,0.0,0.0
20723,pH Buffer Solution 7.00,0.0,0.0,0.0,0.0,0.0
20724,packing tapr dispenser,0.0,0.0,0.0,0.0,0.0
20725,"signboard painter's colours, modifying tints, ...",0.0,0.0,0.0,0.0,0.0


In [23]:
### Pending quantity from subcontracting order
## Filter status: Partially Received and Material Transferred
## Call here from subcontractiing order 

In [24]:
#Api Get Method - Subcontracting Order
import requests
import pandas as pd
import time 
base_url = 'https://erpv14.electrolabgroup.com/'
endpoint = 'api/resource/Subcontracting Order'
url = base_url + endpoint

params = {
    'fields': '["supplier_name","items.item_code","items.qty","items.received_qty"]',
    'limit_start': 0, 
    'limit_page_length': 1000000000000000,
    'filters': '[["status", "in", ["Partially Received", "Material Transferred"]]]'
}

headers = {
    'Authorization': 'token 3ee8d03949516d0:6baa361266cf807'
}
response = requests.get(url, params=params, headers=headers)

if response.status_code == 200:
    data = response.json()
    print("Fields are correct.")
    so_df = pd.DataFrame(data['data'])
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
    print("Response:", response.json())


Fields are correct.


In [25]:
so_df['SO PENDING QTY'] = so_df['qty'] - so_df['received_qty']
so_df

,supplier_name,item_code,qty,received_qty,SO PENDING QTY
0,Samarth Enterprises,0703 Pump mounting Bracket (Machbow),27.0,0.0,27.0
1,Sacardande Engineers Private Limited,"Vernier Caliper 6"" Mitutoyo (Calibrated)",3.0,0.0,3.0
2,Samarth Enterprises,0204 Bath Lid,2.0,0.0,2.0
3,Samarth Enterprises,0204000022/00 MS,2.0,0.0,2.0
4,Samarth Enterprises,0204000014/00 MS,2.0,0.0,2.0
...,...,...,...,...,...
233,Upstreame Fabrikation Llp,0104000964/03,2000.0,100.0,1900.0
234,S.V.G. Metal Finisher,0215000003/08 SS,40.0,9.0,31.0
235,S.V.G. Metal Finisher,0215 Back Cover SS,40.0,0.0,40.0
236,S.V.G. Metal Finisher,0214 Main Cover Weldment SS,20.0,0.0,20.0


In [26]:
import pandas as pd

# Assuming so_df is already defined and contains the necessary columns
so_df_grouped = so_df.groupby('item_code', as_index=False).agg({
    'SO PENDING QTY': 'sum',
    'supplier_name': ', '.join  
})

empty_row = pd.DataFrame([['', '']], columns=['item_code', 'supplier_name'])  # Adjust columns as necessary

so_df_grouped = pd.concat([so_df_grouped, empty_row], ignore_index=True)

so_df_grouped


,item_code,SO PENDING QTY,supplier_name
0,(OLD) 0903000139/02,9.0,S.V.G. Metal Finisher
1,(OLD) 0913000050/01,20.0,S.V.G. Metal Finisher
2,(OLD) 0913000096/01,10.0,S.V.G. Metal Finisher
3,(OLD) 0913000142/01,10.0,S.V.G. Metal Finisher
4,0104 Spindle Latch,885.0,Microtech Industries
...,...,...,...
212,Slotted Weight ( SS ) 1KG M1 (Calibrated),2.0,Sacardande Engineers Private Limited
213,Slotted Weight ( SS ) 2KG M1 (Calibrated),1.0,Sacardande Engineers Private Limited
214,Slotted Weight ( SS ) 500 GMS M1 (Calibrated),2.0,Sacardande Engineers Private Limited
215,"Vernier Caliper 6"" Mitutoyo (Calibrated)",3.0,Sacardande Engineers Private Limited


In [27]:
# Create the dictionary with desired format
so_df_grouped['Dictionary'] = so_df_grouped.apply(
    lambda row: {f"{supplier.strip()}": row['SO PENDING QTY'] for supplier in row['supplier_name'].split(',')},
    axis=1
)

# Display the updated DataFrame
so_df_grouped


,item_code,SO PENDING QTY,supplier_name,Dictionary
0,(OLD) 0903000139/02,9.0,S.V.G. Metal Finisher,{'S.V.G. Metal Finisher': 9.0}
1,(OLD) 0913000050/01,20.0,S.V.G. Metal Finisher,{'S.V.G. Metal Finisher': 20.0}
2,(OLD) 0913000096/01,10.0,S.V.G. Metal Finisher,{'S.V.G. Metal Finisher': 10.0}
3,(OLD) 0913000142/01,10.0,S.V.G. Metal Finisher,{'S.V.G. Metal Finisher': 10.0}
4,0104 Spindle Latch,885.0,Microtech Industries,{'Microtech Industries': 885.0}
...,...,...,...,...
212,Slotted Weight ( SS ) 1KG M1 (Calibrated),2.0,Sacardande Engineers Private Limited,{'Sacardande Engineers Private Limited': 2.0}
213,Slotted Weight ( SS ) 2KG M1 (Calibrated),1.0,Sacardande Engineers Private Limited,{'Sacardande Engineers Private Limited': 1.0}
214,Slotted Weight ( SS ) 500 GMS M1 (Calibrated),2.0,Sacardande Engineers Private Limited,{'Sacardande Engineers Private Limited': 2.0}
215,"Vernier Caliper 6"" Mitutoyo (Calibrated)",3.0,Sacardande Engineers Private Limited,{'Sacardande Engineers Private Limited': 3.0}


In [28]:
### Group by item_code annd sum pending quanity 

In [29]:
## Dictionary column {item code: {warhouse: pneing quanity and total quanity}}, this will be a column in the the result_df 

In [30]:
result_df = pd.merge(result_df,so_df_grouped , on='item_code', how='outer')

In [31]:
result_df = result_df.fillna({
    'item_code': '', 
    'supplier_name': '',
    'SO PENDING QTY':0,
    'Mahape Quantity': 0,  # 0 for numeric columns
    'QualityControl Quantity': 0,
    'Stock Mahape + QC': 0,
    'Pending Qty': 0,
    'Total': 0,
    'Dictionary':''    
})
# result_df['item_name'] = result_df['item_name'].apply(lambda x: x[:100] if isinstance(x, str) else x)
result_df['Total'] = result_df['Total'] + result_df['SO PENDING QTY']
result_df

,item_code,Mahape Quantity,QualityControl Quantity,Stock Mahape + QC,Pending Qty,Total,SO PENDING QTY,supplier_name,Dictionary
0,,0.0,0.0,0.0,0.0,0.0,0.0,,{'': nan}
1,"""z"" Stand (M.S)",11.0,0.0,11.0,0.0,11.0,0.0,,
2,( 0130000086/02 ),1.0,0.0,1.0,0.0,1.0,0.0,,
3,( 0207000182/00 ),0.0,0.0,0.0,0.0,0.0,0.0,,
4,( R & D) 0115 Square Pillar for Opto,2.0,0.0,2.0,0.0,2.0,0.0,,
...,...,...,...,...,...,...,...,...,...
20723,pH Buffer Solution 4.00,0.0,0.0,0.0,0.0,0.0,0.0,,
20724,pH Buffer Solution 7.00,0.0,0.0,0.0,0.0,0.0,0.0,,
20725,packing tapr dispenser,0.0,0.0,0.0,0.0,0.0,0.0,,
20726,"signboard painter's colours, modifying tints, ...",0.0,0.0,0.0,0.0,0.0,0.0,,


In [32]:
result_df = result_df[result_df['item_code'].notna() & (result_df['item_code'] != '')]
result_df

,item_code,Mahape Quantity,QualityControl Quantity,Stock Mahape + QC,Pending Qty,Total,SO PENDING QTY,supplier_name,Dictionary
1,"""z"" Stand (M.S)",11.0,0.0,11.0,0.0,11.0,0.0,,
2,( 0130000086/02 ),1.0,0.0,1.0,0.0,1.0,0.0,,
3,( 0207000182/00 ),0.0,0.0,0.0,0.0,0.0,0.0,,
4,( R & D) 0115 Square Pillar for Opto,2.0,0.0,2.0,0.0,2.0,0.0,,
5,(0901)EH-01A Power Adapter With Cable,0.0,0.0,0.0,0.0,0.0,0.0,,
...,...,...,...,...,...,...,...,...,...
20723,pH Buffer Solution 4.00,0.0,0.0,0.0,0.0,0.0,0.0,,
20724,pH Buffer Solution 7.00,0.0,0.0,0.0,0.0,0.0,0.0,,
20725,packing tapr dispenser,0.0,0.0,0.0,0.0,0.0,0.0,,
20726,"signboard painter's colours, modifying tints, ...",0.0,0.0,0.0,0.0,0.0,0.0,,


In [33]:
import psycopg2
from psycopg2 import sql, Error
import json

# Database connection
try:
    connection = psycopg2.connect(
        host='localhost',
        port=5432,
        database='postgres',
        user='postgres', 
        password='admin@123'
    )
    
    if connection:
        print("Connected to PostgreSQL")

        # Prepare the cursor
        cursor = connection.cursor()
        
        # Empty the table
        cursor.execute("TRUNCATE TABLE result_df")
        print("Table emptied")
        
        # Prepare the insert query
        insert_query = """INSERT INTO result_df (Dictionary, item_code, so_pending_qty, mahape_quantity, qualitycontrol_quantity, stock_mahape_qc, pending_qty, total)
                          VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"""
        
        # Insert new data
        for index, row in result_df.iterrows():
            supplier_dict = row['Dictionary']
            
            if not isinstance(supplier_dict, dict):
                supplier_dict = None
            
            supplier_dict_json = json.dumps(supplier_dict) 
            
            # Execute insert query
            cursor.execute(insert_query, (
                supplier_dict_json,
                row['item_code'], 
                row['SO PENDING QTY'],
                row['Mahape Quantity'], 
                row['QualityControl Quantity'], 
                row['Stock Mahape + QC'],
                row['Pending Qty'],
                row['Total']
            ))
        
        connection.commit()
        print("Data inserted into PostgreSQL table")
        
except Error as e:
    print(f"Error: {e}")
finally:
    if connection:
        cursor.close()
        connection.close()
        print('Connection closed.')


Connected to PostgreSQL
Table emptied
Data inserted into PostgreSQL table
Connection closed.


In [34]:
result_df.to_excel('test.xlsx')